## Extract features at 3 specified pyramid levels

This notebook follows the same structure as `Extract_feature_CSV.ipynb` but uses a wrapper that extracts features at exactly three user-specified pyramid levels.

Usage: set `csv_path`, `output_csv` and `levels` (exactly 3 integers) then run the extraction loop.

In [ ]:
# Imports and parameters
import os
import pandas as pd
from utils_python.feature_extractor_2D import feature_extraction_2D_three_levels
from utils_python import convert_path_format  # if available in your utils package

# Input CSV (update if needed)
csv_path = '/raid60/individuals/sarah/path-allgland1.csv'
# Output CSV where per-sample feature rows will be appended
output_csv = '/raid60/individuals/sarah/path-allgland1_3levels_features.csv'
# Specify exactly three levels here (e.g. [0,1,2])
levels = [0, 1, 2]

# Read CSV
df_path = pd.read_csv(csv_path)
# Columns used below mirror Extract_feature_CSV.ipynb
tumor_heatmap_list = df_path['TumorMapPath'].to_list()
vessel_list = df_path['VesselMaskPath'].to_list()
crop = list(zip(df_path['StartIdx'].to_list(), df_path['endIdx'].to_list()))
sample_names = df_path['Record ID'].to_list()

# Adjust tumor paths if they are .tif in your CSV (same transformation as original notebook)
tumor_heatmap_list = [i.replace('.tif', '_downsampled.nii.gz') if isinstance(i, str) else i for i in tumor_heatmap_list]
# If convert_path_format exists in your utils_python, uncomment the next lines to convert path formats
# tumor_heatmap_list = [convert_path_format(i, 'windows') if isinstance(i, str) else i for i in tumor_heatmap_list]
# vessel_list = [convert_path_format(i, 'windows') if isinstance(i, str) else i for i in vessel_list]

print(f'Loaded {len(sample_names)} samples from {csv_path}')

### Run extraction loop
Loop over rows and call `feature_extraction_2D_three_levels` for each sample. Results are appended to `output_csv`.

In [ ]:
# Validate levels
if len(levels) != 3:
    raise ValueError('`levels` must contain exactly three integers')

# Process each sample and append a row to the output CSV
for i in range(len(sample_names)):
    sample_name = sample_names[i]
    print('Sample #', i, sample_name)
    if not isinstance(sample_name, str):
        print(f'sample name missing for index {i}, skipping')
        continue

    tumor_path = tumor_heatmap_list[i]
    vessel_path = vessel_list[i]
    z_levels = crop[i]

    # Only proceed if both paths are strings and exist on disk
    if isinstance(tumor_path, str) and isinstance(vessel_path, str):
        if os.path.exists(tumor_path) and os.path.exists(vessel_path):
            # Call the three-level extractor; image_path here is the tumor heatmap (2D extraction)
            try:
                features = feature_extraction_2D_three_levels(tumor_path, levels, output_csv=output_csv, image_id=sample_name, verbose=False)
                # optionally, include z_levels or vessel info in saved CSV by reading/writing the row manually
                # The extractor appends its own fields; to add extra columns we could load the CSV and merge, but keep it simple for now.
                print('  extracted features for', sample_name)
            except Exception as e:
                print('  extraction failed for', sample_name, e)
        else:
            print('  file(s) not found, skipped')
    else:
        print('  missing path(s), skipped')

print('Done')

In [ ]:
# Quick peek at output CSV (if created)
import pandas as pd
from pathlib import Path
if Path(output_csv).exists():
    print('Output CSV rows:', len(pd.read_csv(output_csv)))
    display(pd.read_csv(output_csv).head())
else:
    print('Output CSV not found yet:', output_csv)